In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "13cf196a",
   "metadata": {},
   "source": [
    "# EEE102 Activity 1\n",
    "\n",
    "This activity is an application of bisection method for univariate root-finding."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "242dd356",
   "metadata": {},
   "source": [
    "## Imports"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 36,
   "id": "c5809293",
   "metadata": {},
   "outputs": [],
   "source": [
    "import math as mt\n",
    "\n",
    "import scipy.optimize as spo\n",
    "\n",
    "import math\n",
    "import cmath\n",
    "\n"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "b459de5d",
   "metadata": {},
   "source": [
    "## Scenario"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "9f5fd5c0",
   "metadata": {},
   "source": [
    "You are a power systems engineer tasked to analyze two customer loads and served at a common bus.\n",
    "The bus is at the receiving end of a feeder line extending from the utility system.\n",
    "\n",
    "One of the loads is estimated to be 400 kilowatts at a lagging power factor of 0.85,\n",
    "while the other is rated 300 kilovars at a lagging power factor of 0.75.\n",
    "\n",
    "Records show that the feeder line impedance is rated 1.00 + j2.50 ohms.\n",
    "If it is reasonable to assume that the utility system's RMS voltage hovers around 13.8 kilovolts,\n",
    "at what RMS voltage are the loads served?"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "5b68728e",
   "metadata": {},
   "source": [
    "You are to approach this as a root-finding problem,\n",
    "and so must first derive a function $f\\!\\left(V\\right)$\n",
    "whose root corresponds to the desired quantity."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "5ed56af3",
   "metadata": {},
   "source": [
    "## Bisection"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "9038c957",
   "metadata": {},
   "source": [
    "Construct a Python function `resid_from_x()` that implements $f\\!\\left(V\\right)$.\n",
    "The function takes a single positional argument `x`,\n",
    "representing $V$."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 52,
   "id": "bdbd41f8",
   "metadata": {},
   "outputs": [],
   "source": [
    "def resid_from_x(V):\n",
    "    \"\"\"\n",
    "    Returns the difference (in kV) between the sending-end voltage magnitude\n",
    "    and the specified 13.8 kV nominal level.\n",
    "    \"\"\"\n",
    "    V_rms = V * 1000  # convert kV -> V\n",
    "    I = S_total.conjugate() / V_rms\n",
    "    V_send = V_rms + I * Z_line\n",
    "    return abs(V_send) / 1000 - V_send_nom\n",
    "\n",
    "def find_bracket(func, start=1.0, stop=50.0, step=0.5):\n",
    "    \"\"\"Automatically find a valid [XL, XU] interval where func changes sign.\"\"\"\n",
    "    a = start\n",
    "    while a < stop:\n",
    "        b = a + step\n",
    "        if func(a) * func(b) < 0:\n",
    "            return a, b\n",
    "        a += step\n",
    "    raise RuntimeError(\"No valid bracket found in range.\")\n",
    "\n",
    "# Load 1: 400 kW at 0.85 pf lagging\n",
    "P1 = 400.0\n",
    "pf1 = 0.85\n",
    "Q1 = P1 * math.tan(math.acos(pf1))\n",
    "\n",
    "# Load 2: 300 kVAR at 0.75 pf lagging\n",
    "Q2 = 300.0\n",
    "pf2 = 0.75\n",
    "P2 = Q2 / math.tan(math.acos(pf2))\n",
    "\n",
    "# Combine Total Power loads (convert to VA)\n",
    "S_total = complex(P1 + P2, Q1 + Q2) * 1000\n",
    "\n",
    "# Impedance\n",
    "Z_line = complex(1.0, 2.5)\n",
    "\n",
    "# Nominal sending voltage (kV)\n",
    "V_send_nom = 13.8"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "761b494c",
   "metadata": {},
   "source": [
    "Use bisection to find a root of $f\\!\\left(V\\right)$.\n",
    "Use an iteration budget of 750 and a tolerance of $10^{-7}$.\n",
    "\n",
    "Define the following variables.\n",
    "- `XL`, to store the lower end of the search interval\n",
    "- `XU`, to store the upper end of the search interval\n",
    "- `MAX_ITERS`, to store the iteration budget\n",
    "- `X_TOL` to store the tolerance\n",
    "\n",
    "Run\n",
    "[`scipy.optimize.bisect()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html)\n",
    "such that you get the (approximate) root as well as information concerning the run.\n",
    "Store the root and the information in variables `p_bs` and `p_bs_info`, respectively.\n",
    "Make sure to verify if the returned root is indeed a root."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 48,
   "id": "e312d37e",
   "metadata": {},
   "outputs": [],
   "source": [
    "XL, XU = find_bracket(resid_from_x)\n",
    "X_TOL = 1e-7\n",
    "MAX_ITERS = 750\n",
    "\n",
    "p_bs, p_bs_info = spo.bisect(\n",
    "    resid_from_x,\n",
    "    XL,\n",
    "    XU,\n",
    "    xtol=X_TOL,\n",
    "    maxiter=MAX_ITERS,\n",
    "    full_output=True\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "31afdfab",
   "metadata": {},
   "source": [
    "## Results\n",
    "\n",
    "Do not modify nor remove the following code cell."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 49,
   "id": "83df63f6",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Finding a root in the interval [13.5, 14.0]:\n",
      "      converged: True\n",
      "           flag: converged\n",
      " function_calls: 25\n",
      "     iterations: 23\n",
      "           root: 13.645041286945343\n",
      "         method: bisect\n",
      "Residual value: -5.176121753436291e-08\n"
     ]
    }
   ],
   "source": [
    "print(f\"Finding a root in the interval [{XL}, {XU}]:\")\n",
    "print(p_bs_info)\n",
    "print(f\"Residual value: {resid_from_x(p_bs)}\")\n",
    "\n"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "16ebcfa6",
   "metadata": {},
   "source": [
    "## Submission instructions"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "84dd36b9",
   "metadata": {},
   "source": [
    "Download this notebook file,\n",
    "and save with a filename following the pattern\n",
    "`ACT-01_<section>_<ID number>`,\n",
    "where the section is as reflected in your Google Classroom.\n",
    "For example, if your ID number is 2013-0024\n",
    "and you are enrolled to the M34W12 class,\n",
    "then your notebook should be named `ACT-01_M34W12_2013-0024.ipynb`.\n",
    "Submit your notebook via the classwork platform for this activity in Google Classroom.\n",
    "\n",
    "Lastly, the use of AI tools to answer this exam is not prohibited,\n",
    "but it is of ethical interest to disclose such use.\n",
    "This is in line with the\n",
    "[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).\n",
    "As such, please include a brief statement (in a private comment to this classwork)\n",
    "declaring which and how AI tools are used in your work."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "f3bf61ca",
   "metadata": {},
   "source": [
    "*Last updated by Christian Cahig on 2025-10-01*"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d4401784-a654-420a-b556-23dd276a87b0",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "2881bde8-4b9a-45b3-baef-af398dbb95be",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.13.7"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


NameError: name 'null' is not defined